In [ ]:
import os, shutil
train_dir = '../train'
validation_dir = '../validation'
test_dir = '../test'

In [ ]:
from keras.utils import image_dataset_from_directory

IMG_SIZE = 150

train_dataset = image_dataset_from_directory(train_dir, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE))
validation_dataset = image_dataset_from_directory(validation_dir, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE))
test_dataset = image_dataset_from_directory(test_dir, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE))

#### Dividir o dataset ####
def get_dataset_size(dataset):
    return sum(1 for _ in dataset)

train_size = get_dataset_size(train_dataset)
validation_size = get_dataset_size(validation_dataset)

part_train_size = train_size // 4
part_validation_size = validation_size // 4

def split_dataset(dataset, part_size):
    parts = []
    for i in range(4):
        parts.append(dataset.skip(i * part_size).take(part_size))
    return parts

train_parts = split_dataset(train_dataset, part_train_size)
validation_parts = split_dataset(validation_dataset, part_validation_size)

train_dataset_1, train_dataset_2, train_dataset_3, train_dataset_4 = train_parts
validation_dataset_1, validation_dataset_2, validation_dataset_3, validation_dataset_4 = validation_parts

print(f"Train dataset parts sizes: {[get_dataset_size(part) for part in train_parts]}")
print(f"Validation dataset parts sizes: {[get_dataset_size(part) for part in validation_parts]}")


In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

conv_base = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
conv_base.trainable = False

In [ ]:
from tensorflow import keras 
import numpy as np

def get_features_and_labels(dataset): 
    all_features = []
    all_labels = []

    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg19.preprocess_input(images) 
        features = conv_base.predict(preprocessed_images) 
        all_features.append(features)
        all_labels.append(labels)

    return np.concatenate(all_features), np.concatenate(all_labels)

In [ ]:
#Dividimos o dataset em quatro partes
#1/4
train_features_1, train_labels_1 = get_features_and_labels(train_dataset_1)
val_features_1, val_labels_1 = get_features_and_labels(validation_dataset_1)

#2/4
train_features_2, train_labels_2 = get_features_and_labels(train_dataset_2)
val_features_2, val_labels_2 = get_features_and_labels(validation_dataset_2)

#3/4
train_features_3, train_labels_3 = get_features_and_labels(train_dataset_3)
val_features_3, val_labels_3 = get_features_and_labels(validation_dataset_3)

#4/4
train_features_4, train_labels_4 = get_features_and_labels(train_dataset_4)
val_features_4, val_labels_4 = get_features_and_labels(validation_dataset_4)

In [ ]:
import numpy as np

save_path = 'features/ModelT_transferLearning_featureExtraction_WithoutDataAumentation/'

np.save(save_path + 'train_features_dataset1.npy', train_features_1)
np.save(save_path + 'train_features_dataset2.npy', train_features_2)
np.save(save_path + 'train_features_dataset3.npy', train_features_3)
np.save(save_path + 'train_features_dataset4.npy', train_features_4)

np.save(save_path + 'val_features_dataset1.npy', val_features_1)
np.save(save_path + 'val_features_dataset2.npy', val_features_2)
np.save(save_path + 'val_features_dataset3.npy', val_features_3)
np.save(save_path + 'val_features_dataset4.npy', val_features_4)

np.save(save_path + 'train_features_dataset1.npy', train_labels_1)
np.save(save_path + 'train_features_dataset2.npy', train_labels_2)
np.save(save_path + 'train_features_dataset3.npy', train_labels_3)
np.save(save_path + 'train_features_dataset4.npy', train_labels_4)

np.save(save_path + 'train_features_dataset1.npy', val_labels_1)
np.save(save_path + 'train_features_dataset2.npy', val_labels_2)
np.save(save_path + 'train_features_dataset3.npy', val_labels_3)
np.save(save_path + 'train_features_dataset4.npy', val_labels_4)

In [ ]:
conv_base.summary()

In [ ]:
from tensorflow import keras
from keras import layers
from tensorflow.keras.regularizers import l2

inputs = keras.Input(shape=(4, 4, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.7)(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = keras.Model(inputs, outputs)

In [ ]:
#A partir deste bloco iremos treinar o modelo para os sub datasets
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-5, weight_decay=1e-2),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

In [ ]:
history = model.fit(train_features_1, train_labels_1,epochs=120, batch_size=32, validation_data=(val_features_1, val_labels_1),callbacks=[early_stopping,reduce_lr])

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc,'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss,'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=1e-1),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7)

history = model.fit(train_features_2, train_labels_2,epochs=120, batch_size=32, validation_data=(val_features_2, val_labels_2),callbacks=[early_stopping,reduce_lr])

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc,'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss,'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=0.5),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7)

history = model.fit(train_features_3, train_labels_3,epochs=120, batch_size=32, validation_data=(val_features_3, val_labels_3),callbacks=[early_stopping,reduce_lr])

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc,'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss,'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=0.5),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7)

history = model.fit(train_features_4, train_labels_4,epochs=120, batch_size=32, validation_data=(val_features_4, val_labels_4),callbacks=[early_stopping,reduce_lr])

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc,'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss,'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
from keras import models
inputs = keras.Input(shape=(150, 150, 3))
x = keras.applications.vgg19.preprocess_input(inputs) 
x = conv_base(x)
outputs = model(x)
full_model = keras.Model(inputs, outputs)

full_model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=0.5),metrics=['accuracy'])

In [ ]:
full_model.save('ModelT_transferLearning_featureExtraction_WithoutDataAumentation.h5')
model.save('ModelT_transferLearning_featureExtraction_WithoutDataAumentation_OnlyClassification.h5')

In [ ]:
from tensorflow import keras

loaded_model = keras.models.load_model('models/ModelT_transferLearning_featureExtraction_WithoutDataAumentation.h5')

val_loss, val_acc = loaded_model.evaluate(validation_dataset)
print('val_acc:', val_acc)

In [ ]:
val_loss, val_acc = loaded_model.evaluate(test_dataset) 
print('val_acc:', val_acc)